In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
# Load the data
data_root_path = '/home/ec2-user/Neural_Network/Neural_Network/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test').T

print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs].T
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1).T
y_train = np.delete(y_train,indexs)

{'truck': 9, 'dog': 5, 'automobile': 1, 'bird': 2, 'frog': 6, 'airplane': 0, 'deer': 4, 'cat': 3, 'ship': 8, 'horse': 7}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
print(X_validation.shape)
print(X_test.shape)
arr = np.arange(X_train.shape[0])
np.random.shuffle(arr)
print(arr)

(5000, 3072)
(10000, 3072)
[24066 35091   539 ...,  1533 39374 12473]


In [6]:
def get_batch(X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        m = X.shape[0]
        start_index = np.random.randint(0, m - batch_size)
        X_batch = X[start_index:(start_index + batch_size), :]
        y_batch = y[start_index:(start_index + batch_size)]
        
        return X_batch, y_batch

In [7]:
logdir = 'tf_logs/summary6'
iteration = 20000
batch_size = 128
learning_rate = 0.01

In [8]:
# try use a function to create a layer
# def create_conv(filters, input_layer):
#     conv = tf.layers.conv2d(inputs=input_layer, filters=filters, kernel_size=3, strides=1, 
#                                     padding = 'SAME', activation = tf.nn.relu, name='conv')
#     pool = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding='SAME', name='pool')
#     return pool

In [9]:
def save_variable(var, name):
    with tf.name_scope(name) as scope:
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        stddev = tf.sqrt(tf.reduce_mean(var - mean))
        tf.summary.scalar('stddev', stddev)
        tf.summary.histogram('histgram', var)

In [10]:
with tf.device('/gpu:0'):
    with tf.name_scope('input') as scope:
        X = tf.placeholder(tf.float32, shape = (None, 3072))
        y = tf.placeholder(tf.int32, shape = (None))
        dropout_rate = tf.placeholder(tf.float32, shape=())
        training = tf.placeholder(tf.bool)
        input_layer = tf.reshape(X, [-1, 32, 32, 3])

    with tf.name_scope('conv1') as scope:
        conv1 = tf.layers.conv2d(inputs=input_layer, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv1')
#         weight1 = tf.contrib.layers.flatten(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1'))
        weight1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1')[0]
        batch_norm1 = tf.layers.batch_normalization(conv1, axis = 1, name='batch_norm1')
        dropout1 = tf.layers.dropout(batch_norm1,rate=0.1, training=training, name='dropout1')
        pool1 = tf.layers.max_pooling2d(inputs=dropout1, pool_size=2, strides=2, padding='SAME', name='pool1')

    with tf.name_scope('conv2') as scope:
        conv2 = tf.layers.conv2d(inputs=pool1, filters=128, kernel_size=3, strides=1, 
                            padding = 'SAME', activation = tf.nn.relu, name='conv2')
        weight2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv2')[0]
        batch_norm2 = tf.layers.batch_normalization(conv2, axis = 1, name='batch_norm2')
        dropout2 = tf.layers.dropout(batch_norm2,rate=0.2, training=training, name='dropout2')
        pool2 = tf.layers.max_pooling2d(inputs=dropout2, pool_size=2, strides=2, padding='SAME', name='pool2')  
        
    with tf.name_scope('conv3') as scope:
        conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=3, strides=1, 
                            padding = 'SAME', activation = tf.nn.relu, name='conv3')
        weight3 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv3')[0]
        batch_norm3 = tf.layers.batch_normalization(conv3, axis = 1, name='batch_norm3')
        dropout3 = tf.layers.dropout(batch_norm3,rate=0.2, training=training, name='dropout3')
        pool3 = tf.layers.max_pooling2d(inputs=dropout3, pool_size=2, strides=2, padding='SAME', name='pool3')

    with tf.name_scope('fc') as scope:
        pool3_flat = tf.reshape(pool3, [-1, 4*4*128], name='pool3_flat')
        dropout_fc1 = tf.layers.dropout(pool3_flat, rate=0.2, training=training, name='dropout_fc1')
        
        dense1 = tf.layers.dense(dropout_fc1, units = 8*128, activation = tf.nn.relu, name='dense1')
        batch_norm4 = tf.layers.batch_normalization(dense1, axis = 1, name='batch_norm4')
        dropout_fc2 = tf.layers.dropout(batch_norm4, rate=0.2, training=training, name='dropout_fc2')
        
        dense2 = tf.layers.dense(dropout_fc2, units = 128, activation = tf.nn.relu, name='dense2') 
        batch_norm5 = tf.layers.batch_normalization(dense2, axis = 1, name='batch_norm5')
        dropout_fc3 = tf.layers.dropout(batch_norm5, rate=0.2, training=training, name='dropout_fc3')


    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(dropout_fc3, units = 10, name='logits')
        weight_logits = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'logits')[0]

    with tf.name_scope('loss') as scope:
        softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name = 'softmax')
        loss = tf.reduce_mean(softmax) 

    with tf.name_scope('train') as scope:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    with tf.name_scope('eval') as scope:
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
     
    with tf.name_scope('summary') as scope:
        save_variable(weight1,'weight1')
        save_variable(weight2,'weight2')
        save_variable(weight3,'weight3')
        save_variable(weight_logits,'weight_logits')
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(logdir + '/train', tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logdir + '/test', tf.get_default_graph())

    with tf.name_scope('init') as scope:
        init = tf.global_variables_initializer()
    

In [11]:
print(weight_logits)

<tf.Variable 'logits/kernel:0' shape=(128, 10) dtype=float32_ref>


In [12]:
def train(X_train, y_train, X_validation, y_validation, iteration, X_test):

    config = tf.ConfigProto()
    config.allow_soft_placement = True
    
    with tf.Session(config=config) as sess:
        sess.run(init)
        for step in range(iteration):
            X_batch, y_batch = get_batch(X_train, y_train, batch_size)
            sess.run(optimizer, feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:True})
                
            if step % 100 == 0:
                summary, loss_, acc = sess.run([merged, loss, accuracy],
                                               feed_dict={X: X_validation, y:y_validation, dropout_rate:0.3, training:False})
                test_writer.add_summary(summary, step)
                print('###################################')
                print('validation! after '+ str(step) + ' iterations' + 
                              ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
                        
                summary, loss_, acc = sess.run([merged, loss, accuracy], 
                                               feed_dict={X: X_batch, y:y_batch,dropout_rate:0.3, training:False})
                train_writer.add_summary(summary, step)
                        
                print('training! after '+ str(step) + ' iterations' + 
                          ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
            
        y_labels = logits.eval(feed_dict={X: X_test, dropout_rate:0.3, training:False})
    return y_labels.T

In [ ]:
y_predicted = train(X_train, y_train, X_validation, y_validation, iteration, X_test)

###################################
validation! after 0 iterations the loss is 2.29992, the accuracy is 0.1264
training! after 0 iterations the loss is 2.29078, the accuracy is 0.125
###################################
validation! after 100 iterations the loss is 2.28141, the accuracy is 0.1958
training! after 100 iterations the loss is 2.2803, the accuracy is 0.242188
###################################
validation! after 200 iterations the loss is 2.2361, the accuracy is 0.2596
training! after 200 iterations the loss is 2.23018, the accuracy is 0.3125
###################################
validation! after 300 iterations the loss is 2.1226, the accuracy is 0.2708
training! after 300 iterations the loss is 2.12919, the accuracy is 0.226562
###################################
validation! after 400 iterations the loss is 2.06897, the accuracy is 0.2796
training! after 400 iterations the loss is 2.08469, the accuracy is 0.3125
###################################
validation! after 500 iterat

###################################
validation! after 4400 iterations the loss is 1.35276, the accuracy is 0.5142
training! after 4400 iterations the loss is 1.15154, the accuracy is 0.59375
###################################
validation! after 4500 iterations the loss is 1.34465, the accuracy is 0.5298
training! after 4500 iterations the loss is 1.40207, the accuracy is 0.5625
###################################
validation! after 4600 iterations the loss is 1.32959, the accuracy is 0.5246
training! after 4600 iterations the loss is 1.24594, the accuracy is 0.539062
###################################
validation! after 4700 iterations the loss is 1.35922, the accuracy is 0.5214
training! after 4700 iterations the loss is 1.21305, the accuracy is 0.570312
###################################
validation! after 4800 iterations the loss is 1.39865, the accuracy is 0.5076
training! after 4800 iterations the loss is 1.38346, the accuracy is 0.484375
###################################
validat

###################################
validation! after 8700 iterations the loss is 1.11035, the accuracy is 0.5952
training! after 8700 iterations the loss is 0.885178, the accuracy is 0.664062
###################################
validation! after 8800 iterations the loss is 1.05584, the accuracy is 0.6186
training! after 8800 iterations the loss is 0.945444, the accuracy is 0.65625
###################################
validation! after 8900 iterations the loss is 1.1253, the accuracy is 0.5944
training! after 8900 iterations the loss is 0.804628, the accuracy is 0.71875
###################################
validation! after 9000 iterations the loss is 1.05858, the accuracy is 0.6228
training! after 9000 iterations the loss is 1.01506, the accuracy is 0.617188
###################################
validation! after 9100 iterations the loss is 1.17213, the accuracy is 0.5816
training! after 9100 iterations the loss is 0.966521, the accuracy is 0.703125
###################################
val

###################################
validation! after 13000 iterations the loss is 0.898122, the accuracy is 0.6768
training! after 13000 iterations the loss is 0.894362, the accuracy is 0.664062
###################################
validation! after 13100 iterations the loss is 0.960124, the accuracy is 0.6586
training! after 13100 iterations the loss is 0.921074, the accuracy is 0.640625
###################################
validation! after 13200 iterations the loss is 0.865287, the accuracy is 0.6938
training! after 13200 iterations the loss is 0.562411, the accuracy is 0.828125
###################################
validation! after 13300 iterations the loss is 0.85988, the accuracy is 0.6942
training! after 13300 iterations the loss is 0.782343, the accuracy is 0.734375
###################################
validation! after 13400 iterations the loss is 0.840324, the accuracy is 0.7026
training! after 13400 iterations the loss is 0.795636, the accuracy is 0.726562
#####################

###################################
validation! after 17300 iterations the loss is 0.752096, the accuracy is 0.7302
training! after 17300 iterations the loss is 0.629569, the accuracy is 0.742188
###################################
validation! after 17400 iterations the loss is 0.719896, the accuracy is 0.7438
training! after 17400 iterations the loss is 0.563058, the accuracy is 0.796875
###################################
validation! after 17500 iterations the loss is 0.834337, the accuracy is 0.7028
training! after 17500 iterations the loss is 0.447383, the accuracy is 0.835938
###################################
validation! after 17600 iterations the loss is 0.751479, the accuracy is 0.733
training! after 17600 iterations the loss is 0.498634, the accuracy is 0.820312
###################################
validation! after 17700 iterations the loss is 0.720958, the accuracy is 0.7418
training! after 17700 iterations the loss is 0.632623, the accuracy is 0.789062
#####################

In [ ]:
# save_predictions('ans1-yz3065', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-yz3065.npy')
print(loaded_y.shape)
loaded_y[:10]